In [ ]:
import os
import requests
import zipfile
import io
import pandas as pd
from datetime import datetime

symbol = "BNBUSDC"
interval = "1m"
start_year, start_month = 2024, 1
end_year, end_month = 2025, 9
base_url = "https://data.binance.vision/data/spot/monthly/klines"
output_file = f"{symbol}-{interval}-2years.csv"

all_data = []

def convert_timestamp(ts, year):
    # Binance switched to microsecond timestamps in 2025
    unit = 'ms' if year < 2025 else 'us'
    return pd.to_datetime(ts, unit=unit).dt.strftime('%d.%m.%Y %H:%M')

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        if (year == start_year and month < start_month) or (year == end_year and month > end_month):
            continue

        mm = f"{month:02d}"
        file_name = f"{symbol}-{interval}-{year}-{mm}.zip"
        url = f"{base_url}/{symbol}/{interval}/{file_name}"

        print(f"Downloading: {url}")
        r = requests.get(url)

        if r.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(r.content)) as z:
                csv_name = file_name.replace(".zip", ".csv")
                z.extract(csv_name)
                df = pd.read_csv(csv_name, header=None)
                df.columns = [
                    "open_time", "open", "high", "low", "close",
                    "volume", "close_time", "quote_asset_volume",
                    "number_of_trades", "taker_buy_base_volume",
                    "taker_buy_quote_volume", "ignore"
                ]
                df["timestamp"] = convert_timestamp(df["open_time"], year)
                all_data.append(df)
                os.remove(csv_name)
        else:
            print(f"⚠️ Missing data for {year}-{mm}")

df_all = pd.concat(all_data, ignore_index=True)

df_all.to_csv(output_file, index=False)
print(f"\n✅ Saved: {output_file}  ({len(df_all)} rows)")
